In [ ]:
%store -r
import sys,json
!{sys.executable} -m pip install kfp==1.4.0 kfp-server-api==1.2.0 --user >/dev/null

### Importing libraries

In [ ]:
import os, json
import kfp
from kfp import components
from dkube.sdk import *

### Loading Components

In [ ]:
dkube_preprocessing_op = components.load_component_from_file("/mnt/dkube/pipeline/components/preprocess/component.yaml")
dkube_training_op = components.load_component_from_file("/mnt/dkube/pipeline/components/training/component.yaml")
dkube_serving_op  = components.load_component_from_file("/mnt/dkube/pipeline/components/serving/component.yaml")

In [ ]:
api = DkubeApi(token=os.getenv("DKUBE_USER_ACCESS_TOKEN",TOKEN))
client = kfp.Client(host=os.getenv("KF_PIPELINES_ENDPOINT"),existing_token=os.getenv("DKUBE_USER_ACCESS_TOKEN",TOKEN),namespace=DKUBEUSERNAME)

### Pipeline Inputs

In [ ]:
training_program = 'titanic'
if (DATA_SOURCE == 'local' or DATA_SOURCE == 'aws_s3') and INPUT_TRAIN_TYPE == 'retraining':
    input_training_dataset = mm_name+'-groundtruth'
elif DATA_SOURCE == 'sql':
    input_training_dataset = 'titanic-data-sql'
else:
    input_training_dataset = 'titanic-data'

## Preprocessing stage inputs
preprocessing_script =f"pip3 install pymysql --user;python titanic/preprocessing.py --data_source {DATA_SOURCE} --train_type {INPUT_TRAIN_TYPE} --monitor_name {MONITOR_NAME} --user {DKUBEUSERNAME}"
input_dataset_mount = ['/data']
output_dataset = 'titanic-training-data'
output_mount_path = ['/train-data']

## Training stage inputs
training_script = f"python titanic/training.py --data_source {DATA_SOURCE}"
model_name = 'titanic-model'
output_model_mount = "/model"

### Pipeline Definition

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-titanic-pl',
    description='sample titanic pipeline with dkube components'
)
def titanic_pipeline(token):
    
    preprocessing = dkube_preprocessing_op(
                                    auth_token=str(token),
                                    container=json.dumps({"image":"docker.io/ocdr/dkube-datascience-tf-cpu:v2.0.0-3"}),
                                    program=str(training_program),
                                    datasets = json.dumps([str(input_training_dataset)]),
                                    input_dataset_mounts = json.dumps(input_dataset_mount),
                                    run_script=str(preprocessing_script),
                                    outputs=json.dumps([str(output_dataset)]),
                                    output_mounts=json.dumps(output_mount_path)).set_display_name("data-generation")
    
    train       = dkube_training_op(auth_token = token,
                                    container = json.dumps({"image":"docker.io/ocdr/dkube-datascience-tf-cpu:v2.0.0-3"}),
                                    framework="tensorflow", version="2.0.0",
                                    program=str(training_program), 
                                    run_script=str(training_script),
                                    datasets=json.dumps([str(output_dataset)]), outputs=json.dumps([str(model_name)]),
                                    input_dataset_mounts=json.dumps(output_mount_path),
                                    output_mounts=json.dumps([str(output_model_mount)]),
                                    envs='[{"EPOCHS": "1"}]').after(preprocessing)
    
    serving     = dkube_serving_op(auth_token = token,model = train.outputs['artifact'], device='cpu', 
                                    serving_image=json.dumps({"image":"ocdr/tensorflowserver:2.0.0"}),
                                    transformer_image=json.dumps({"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}),
                                    transformer_project=str(training_program),
                                    transformer_code='titanic/transformer.py').after(train)

In [ ]:
client.create_run_from_pipeline_func(titanic_pipeline, arguments={'token':TOKEN})